<a href="https://colab.research.google.com/github/Raghav2608/5CCSAMLF-CW_1/blob/main/Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd



In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split,KFold
from sklearn.metrics import r2_score,root_mean_squared_error
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel,r_regression
from xgboost import XGBRegressor
from sklearn.kernel_ridge import KernelRidge
from lightgbm import LGBMRegressor
from GradientBagging import BaggedGradientBoosting,AdaptiveBaggingGBT
from kernelbagging import BaggedKernelRidge
from sklearn.linear_model import LinearRegression

In [2]:
data = pd.read_csv("CW1_train.csv")

In [ ]:
figure, axis = plt.subplots(31,1,figsize = (8,31*4),tight_layout=True)

for count, value in enumerate(data.columns):
    axis[count].hist(data[value],density=True,bins=100)
    axis[count].set_title(value)

plt.plot() 

In [276]:
print(data["outcome"].mean())
print(data["outcome"].std())

-4.97908145098559
12.72222354244085


In [ ]:
# using label encoder to encode categorical variables 

le1 = LabelEncoder()
le2 = LabelEncoder()
le3 = LabelEncoder()

data["cut"] = le1.fit_transform(data.cut.values)
data["color"] = le2.fit_transform(data.color.values)
data["clarity"] = le3.fit_transform(data.clarity.values)

In [ ]:
figure, axis = plt.subplots(31,1,figsize = (8,31*4),tight_layout=True)

for count, value in enumerate(data.columns):
    axis[count].scatter(data[value],data["outcome"])
    axis[count].set_title(value)
    

plt.show()
plt.savefig("Feature_with_outcome_Plots.png")

In [316]:
import math
import random 
import numpy as np

output = data["outcome"]
mean = output.mean()
std = output.std()
keys = []
random.seed = 42

for i in range(len(output)):
    rand = random.randint(1,100)
    if rand <= 30:
        if abs(output.iloc[i] - mean) >= std:
            keys.append(i)
    else:
        keys.append(i)
    
        

len(keys)


8055

In [ ]:
train = data.iloc[keys]
test = data.drop(index=keys)

In [3]:
data["ppd"]=data["x"]*data["y"]*data["z"]

In [4]:
r_regression(data[["ppd","depth"]],data["outcome"])

array([ 0.00382161, -0.41107324])

In [5]:
low_dim_data = data[["depth","a1","a2","a3","a4","a5","b1","b3","price","table"]]
low_dim_data = (low_dim_data-low_dim_data.mean())/low_dim_data.std()
output = data["outcome"]


In [59]:
print(output.head())
print(low_dim_data.head())

0   -26.701232
1     6.548093
2     6.612562
3    -5.073562
4   -14.436557
Name: outcome, dtype: float64
      depth        a1        a2        a3        a4        a5        b1  \
0  0.386072  0.712126 -0.417961 -0.818125 -0.004256 -0.133121  1.066741   
1 -0.872995  0.505764 -1.613736  0.397700 -1.558762 -1.702437 -0.013376   
2 -0.733098  0.165450 -1.641581 -0.528798 -1.282223 -0.632968  1.117114   
3 -0.383358  1.649759 -0.069824 -0.565831 -0.932573 -0.807628  1.045854   
4  0.456020 -1.620114  1.068077  1.564657  1.462771  1.248267  1.728594   

         b3     price     table  
0 -0.771892  1.024563 -0.653020  
1 -1.417874 -0.764609  0.682072  
2  1.291506 -0.649645  0.237041  
3 -1.481676 -0.355003  0.237041  
4  0.964502 -0.294349 -1.543082  


In [13]:
low_dim_data.head()

,depth,a1,a2,a3,a4,a5,b1,b3,price,table
0,0.386072,0.712126,-0.417961,-0.818125,-0.004256,-0.133121,1.066741,-0.771892,1.024563,-0.653020
1,-0.872995,0.505764,-1.613736,0.397700,-1.558762,-1.702437,-0.013376,-1.417874,-0.764609,0.682072
2,-0.733098,0.165450,-1.641581,-0.528798,-1.282223,-0.632968,1.117114,1.291506,-0.649645,0.237041
3,-0.383358,1.649759,-0.069824,-0.565831,-0.932573,-0.807628,1.045854,-1.481676,-0.355003,0.237041
4,0.456020,-1.620114,1.068077,1.564657,1.462771,1.248267,1.728594,0.964502,-0.294349,-1.543082


In [6]:
X_train_t, X_test_t, y_train_t, y_test_t = train_test_split(low_dim_data,
    output, test_size=0.1, random_state=42)

In [105]:
import numpy as np 
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Initialize the model
model =  XGBRegressor(n_estimators=2000,max_depth=3,max_bin=100,reg_alpha=100,
                      booster="gbtree",tree_method="hist",grow_policy="lossguide",
                      eta=0.007,subsample=0.8)


# Store accuracies
accuracies = []
train_accuracies = []
train_variance = []
test_variance = []
train_subsets = None
test_subsets = None

X = low_dim_data.to_numpy()
y = output.to_numpy()
rmse_data = []
i = 0
# Perform K-Fold Cross-Validation
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Train the model
    model.fit(X_train, 
    y_train,
    eval_set=[(X_test, y_test)],
    verbose=True
    )

    train_variance.append(y_train.std())
    test_variance.append(y_test.std())

    # Predict
    y_pred_train = model.predict(X_train)
    y_pred = model.predict(X_test)

    rmse_data.append(root_mean_squared_error(y_test,y_pred))
    

    i = i + 1
    # Evaluate

    acc = r2_score(y_test, y_pred)
    accuracies.append(acc)
    

# Print the average accuracy

print(f"Average Accuracy: {np.mean(accuracies):.4f}")
print(accuracies)
print()
print(f"Average Accuracy: {np.mean(train_accuracies):.4f}")
print(train_accuracies)
print(f"Average Accuracy: {np.mean(rmse_data):.4f}")
print(rmse_data)

[0]	validation_0-rmse:12.55948
[1]	validation_0-rmse:12.52670
[2]	validation_0-rmse:12.49388
[3]	validation_0-rmse:12.46075
[4]	validation_0-rmse:12.42876
[5]	validation_0-rmse:12.39718
[6]	validation_0-rmse:12.36626
[7]	validation_0-rmse:12.33520
[8]	validation_0-rmse:12.30487
[9]	validation_0-rmse:12.27445
[10]	validation_0-rmse:12.24454
[11]	validation_0-rmse:12.21473
[12]	validation_0-rmse:12.18586
[13]	validation_0-rmse:12.15677
[14]	validation_0-rmse:12.12824
[15]	validation_0-rmse:12.10023
[16]	validation_0-rmse:12.07228
[17]	validation_0-rmse:12.04474
[18]	validation_0-rmse:12.01741
[19]	validation_0-rmse:11.98977
[20]	validation_0-rmse:11.96261
[21]	validation_0-rmse:11.93613
[22]	validation_0-rmse:11.90944
[23]	validation_0-rmse:11.88397
[24]	validation_0-rmse:11.85840
[25]	validation_0-rmse:11.83312
[26]	validation_0-rmse:11.80801
[27]	validation_0-rmse:11.78314
[28]	validation_0-rmse:11.75860
[29]	validation_0-rmse:11.73408
[30]	validation_0-rmse:11.70978
[31]	validation_0-

c:\Users\HP\OneDrive\Documents\GitHub\5CCSAMLF-CW_1\.venv\lib\site-packages\numpy\_core\fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\HP\OneDrive\Documents\GitHub\5CCSAMLF-CW_1\.venv\lib\site-packages\numpy\_core\_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
